In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
training = [("The bear mauled the coyote".split(), ["DET", "NN", "V", "DET", "NN"]),
            ("Smokers despise themselves some would say, not me though".split(), ["NN", "V", "DET", "DET", "V", "V", "DET", "NN", "DET"])]

In [3]:
def word_wise_indexing(text, text_index):
    sequence = text
    element_index = [text_index[word_element] for word_element in sequence]
    return torch.tensor(element_index, dtype=torch.long)

In [4]:
training_data_index = {}

In [5]:
for sentence, tags in training:
    for word in sentence:
        if word not in training_data_index:
            training_data_index[word] = len(training_data_index)

print(training_data_index)

tag_index = {"DET": 0, "NN": 1, "V": 2}

embedding_dims = 6
hi_hidden_dims = 6

{'The': 0, 'bear': 1, 'mauled': 2, 'the': 3, 'coyote': 4, 'Smokers': 5, 'despise': 6, 'themselves': 7, 'some': 8, 'would': 9, 'say,': 10, 'not': 11, 'me': 12, 'though': 13}


In [6]:
class LSTMTagger(nn.Module):

    def __init__(self, emb_dim, hi_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hi_dim = hi_dim

        self.word_embeddings = nn.Embedding(vocab_size, emb_dim)

        self.lstm_model = nn.LSTM(emb_dim, hi_dim)

        self.transform_into_tag_space = nn.Linear(hi_dim, tagset_size)
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        
        # The view() function below reshapes the tensor to satisfy the dimension perameters of the lstm function (sentence length, batch size, embedding dimensions).
        model_output, _ = self.lstm_model(embeds.view(len(sentence), 1, -1)) 

        model_output = model_output.view(len(sentence), self.hi_dim)
        
        # The view() function is used again prior to calling the 2D linear transformation function designed to map the LSTM model's hidden state to tag space.
        tag_space = self.transform_into_tag_space(model_output) # Returns a 2D tensor

        # The log_softmax() function converts the model output score into log-probabilities for all elements in the tag space/original sequence (dim=1).
        scores = F.log_softmax(tag_space, dim=1)

        return scores




In [7]:
lstm_model_run_one = LSTMTagger(embedding_dims, hi_hidden_dims, len(training_data_index), len(tag_index))
fn_loss = nn.NLLLoss()
fn_optimization = optim.SGD(lstm_model_run_one.parameters(), lr=0.1)

In [8]:
with torch.no_grad():
    input_data = word_wise_indexing(training[0][0], training_data_index)
    scores_for_tags = lstm_model_run_one(input_data)
    
    print(scores_for_tags)

tensor([[-0.9660, -0.9325, -1.4880],
        [-0.8993, -0.9933, -1.5017],
        [-0.9367, -1.0355, -1.3744],
        [-0.9673, -0.9975, -1.3819],
        [-0.9393, -0.9643, -1.4790]])


In [9]:
for epoch in range(300):
    for sentence_sequence, assoc_tags in training:
        
        lstm_model_run_one.zero_grad()

        input_segment = word_wise_indexing(sentence_sequence, training_data_index)

        target_entities = word_wise_indexing(assoc_tags, tag_index)

        seq_sen_score = lstm_model_run_one(input_segment)

        with_fixed_loss_fn = fn_loss(seq_sen_score, target_entities)
        with_fixed_loss_fn.backward()
        fn_optimization.step()
    


In [10]:
with torch.no_grad():
    input_data_post_training = word_wise_indexing(training[0][0], training_data_index)

    post_training_scores = lstm_model_run_one(input_data_post_training)

    print(post_training_scores)

tensor([[-0.0558, -3.6985, -3.5225],
        [-5.6297, -0.0340, -3.5130],
        [-2.4759, -3.9727, -0.1086],
        [-0.0705, -3.5977, -3.2021],
        [-5.9280, -0.0226, -3.9258]])
